In [1]:
import pandas as pd
url_dados = "https://bi.s3.es.gov.br/covid19/MICRODADOS.csv"
dados = pd.read_csv(url_dados, sep=";", encoding="latin1", nrows=10000)

In [14]:
# Schema
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DataNotificacao          10000 non-null  object
 1   DataCadastro             10000 non-null  object
 2   DataDiagnostico          10000 non-null  object
 3   DataColeta_RT_PCR        3042 non-null   object
 4   DataColetaTesteRapido    5162 non-null   object
 5   DataColetaSorologia      161 non-null    object
 6   DataColetaSorologiaIGG   282 non-null    object
 7   DataEncerramento         1588 non-null   object
 8   DataObito                4 non-null      object
 9   Classificacao            10000 non-null  object
 10  Evolucao                 10000 non-null  object
 11  CriterioConfirmacao      10000 non-null  object
 12  StatusNotificacao        10000 non-null  object
 13  Municipio                10000 non-null  object
 14  Bairro                   9885 non-null 

In [2]:
# Verificando as colunas do dataframe
dados.columns

Index(['DataNotificacao', 'DataCadastro', 'DataDiagnostico',
       'DataColeta_RT_PCR', 'DataColetaTesteRapido', 'DataColetaSorologia',
       'DataColetaSorologiaIGG', 'DataEncerramento', 'DataObito',
       'Classificacao', 'Evolucao', 'CriterioConfirmacao', 'StatusNotificacao',
       'Municipio', 'Bairro', 'FaixaEtaria', 'IdadeNaDataNotificacao', 'Sexo',
       'RacaCor', 'Escolaridade', 'Gestante', 'Febre',
       'DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia',
       'Cefaleia', 'ComorbidadePulmao', 'ComorbidadeCardio',
       'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo',
       'ComorbidadeObesidade', 'FicouInternado', 'ViagemBrasil',
       'ViagemInternacional', 'ProfissionalSaude', 'PossuiDeficiencia',
       'MoradorDeRua', 'ResultadoRT_PCR', 'ResultadoTesteRapido',
       'ResultadoSorologia', 'ResultadoSorologia_IGG', 'TipoTesteRapido'],
      dtype='object')

In [5]:
# Análise dos dados
pd.crosstab([dados["Classificacao"], dados["Evolucao"]], dados["FicouInternado"])

FicouInternado                     Ignorado   Não  Não Informado  Sim
Classificacao Evolucao                                               
Confirmados   -                          13   244            567   13
              Cura                       39    32             77    0
              Ignorado                    1    61             74    1
              Óbito pelo COVID-19         0     0              0    3
Descartados   -                           0    94            755    0
              Ignorado                   10  1373            940    3
Suspeito      -                         105  1873           3300   27
              Cura                        0     1              0    0
              Ignorado                    5   180            207    2

In [11]:
# Correlação entre as features
dados.corr().T

,Febre,DificuldadeRespiratoria,Tosse,Coriza,DorGarganta,Diarreia,Cefaleia,ComorbidadePulmao,ComorbidadeCardio,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado
Febre,1.000000,0.003011,0.008210,-0.013835,0.006210,-0.031995,0.007684,-0.000141,-0.000141,-0.000100,-0.000100,-0.000141,-0.000100,-0.001327
DificuldadeRespiratoria,0.003011,1.000000,0.145110,0.083783,0.083045,0.077662,0.087383,0.004258,0.004258,0.003011,0.003011,0.004258,0.003011,-0.007290
Tosse,0.008210,0.145110,1.000000,0.373764,0.307370,0.055631,0.264439,0.011611,-0.002809,0.008210,0.008210,0.011611,0.008210,-0.017750
Coriza,-0.013835,0.083783,0.373764,1.000000,0.257957,0.081036,0.257672,0.010224,0.010224,0.007229,0.007229,0.010224,0.007229,-0.005858
DorGarganta,0.006210,0.083045,0.307370,0.257957,1.000000,0.062936,0.274318,0.008783,-0.006997,0.006210,0.006210,0.008783,0.006210,0.019074
Diarreia,-0.031995,0.077662,0.055631,0.081036,0.062936,1.000000,0.132032,0.004421,0.004421,0.003126,0.003126,0.004421,0.003126,0.011843
Cefaleia,0.007684,0.087383,0.264439,0.257672,0.274318,0.132032,1.000000,0.010867,-0.003771,0.007684,0.007684,0.010867,0.007684,-0.001242
ComorbidadePulmao,-0.000141,0.004258,0.011611,0.010224,0.008783,0.004421,0.010867,1.000000,0.499900,0.707071,0.707071,0.499900,0.707071,-0.001877
ComorbidadeCardio,-0.000141,0.004258,-0.002809,0.010224,-0.006997,0.004421,-0.003771,0.499900,1.000000,0.707071,0.707071,0.499900,0.707071,-0.001877
ComorbidadeRenal,-0.000100,0.003011,0.008210,0.007229,0.006210,0.003126,0.007684,0.707071,0.707071,1.000000,1.000000,0.707071,1.000000,-0.001327


,count,mean,std,min,25%,50%,75%,max
Febre,10000.0,0.0001,0.010000,0.0,0.0,0.0,0.0,1.0
DificuldadeRespiratoria,10000.0,0.9169,0.276047,0.0,1.0,1.0,1.0,1.0
Tosse,10000.0,0.5974,0.490446,0.0,0.0,1.0,1.0,1.0
Coriza,10000.0,0.6568,0.474801,0.0,0.0,1.0,1.0,1.0
DorGarganta,10000.0,0.7217,0.448184,0.0,0.0,1.0,1.0,1.0
Diarreia,10000.0,0.9110,0.284758,0.0,1.0,1.0,1.0,1.0
Cefaleia,10000.0,0.6288,0.483150,0.0,0.0,1.0,1.0,1.0
ComorbidadePulmao,10000.0,0.0002,0.014141,0.0,0.0,0.0,0.0,1.0
ComorbidadeCardio,10000.0,0.0002,0.014141,0.0,0.0,0.0,0.0,1.0
ComorbidadeRenal,10000.0,0.0001,0.010000,0.0,0.0,0.0,0.0,1.0


In [7]:
# features
# transformar as colunas categoricas em numeros
dados.Febre = pd.get_dummies(dados.Febre)
dados.DificuldadeRespiratoria = pd.get_dummies(dados.DificuldadeRespiratoria)
dados.Tosse = pd.get_dummies(dados.Tosse)
dados.Coriza = pd.get_dummies(dados.Coriza)
dados.DorGarganta = pd.get_dummies(dados.DorGarganta)
dados.Diarreia = pd.get_dummies(dados.Diarreia)
dados.Cefaleia = pd.get_dummies(dados.Cefaleia)
dados.ComorbidadePulmao = pd.get_dummies(dados.ComorbidadePulmao)
dados.ComorbidadeCardio = pd.get_dummies(dados.ComorbidadeCardio)
dados.ComorbidadeRenal = pd.get_dummies(dados.ComorbidadeRenal)
dados.ComorbidadeDiabetes = pd.get_dummies(dados.ComorbidadeDiabetes)
dados.ComorbidadeTabagismo = pd.get_dummies(dados.ComorbidadeTabagismo)
dados.ComorbidadeObesidade = pd.get_dummies(dados.ComorbidadeObesidade)

x = dados[['Febre', 'DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia', 'Cefaleia', 'ComorbidadePulmao', 'ComorbidadeCardio', 'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo', 'ComorbidadeObesidade']]
x.DificuldadeRespiratoria.value_counts()

1    9169
0     831
Name: DificuldadeRespiratoria, dtype: int64

In [8]:
# coluna alvo
dados.FicouInternado = pd.get_dummies(dados.FicouInternado)
y = dados["FicouInternado"]
y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: FicouInternado, dtype: uint8

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, random_state=1000, stratify=y)

# Criar o modelo
modelo_rlogistica = LogisticRegression(max_iter=1000)
modelo_rlogistica.fit(x_treino, y_treino)

# previsao
previsoes = modelo_rlogistica.predict(x_teste)

# Faz o predict e testa o acerto
modelo_rlogistica.score(x_teste, y_teste)

0.9825